# Entraînement d'un CNN avec des images de spectrogrammes

Ce notebook entraîne un réseau de neurones convolutionnel (CNN) en utilisant des images de spectrogrammes des vibrations du moteur. Le jeu de données est divisé en ensembles d'entraînement, de validation et de test. Le modèle entraîné est sauvegardé avec un horodatage et une convention de nommage basée sur les performances.

## Importation des bibliothèques (libraries) nécessaires

Nous allons importer les bibliothèques nécessaires pour le chargement des données, le prétraitement et la construction du modèle CNN.

In [2]:
!pip install tensorflow keras matplotlib numpy pandas scikit-learn pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 10.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 9.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 8.9 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import img_to_array, load_img
from datetime import datetime

## Chargement et prétraitement des données

Nous allons charger les images de spectrogrammes depuis le répertoire `data/05_cnn_input`, les prétraiter, puis les diviser en ensembles d'entraînement, de validation et de test.

In [2]:
data_dir = "./../../data/05_cnn_input"

print("Contenu du dossier 05_cnn_input :", os.listdir(data_dir))

image_paths = []
labels = []

for label, category in enumerate(['balourd', 'sain']):
    category_dir = os.path.join(data_dir, category)
    if not os.path.exists(category_dir):
        print(f"Directory does not exist: {category_dir}")
        continue
    print(f"Checking directory: {category_dir}")
    for root, _, files in os.walk(category_dir):
        print(f"Found {len(files)} files in {root}")
        for file in files:
            if file.endswith(".png"):
                image_paths.append(os.path.join(root, file))
                labels.append(label)

print(f"Total directories checked: {len(['balourd', 'sain'])}")
print(f"Total image paths collected: {len(image_paths)}")
print("Sample image paths:", image_paths[:5])
print("Labels distribution:", {label: labels.count(label) for label in set(labels)})

image_paths = np.array(image_paths)
labels = np.array(labels)

Contenu du dossier 05_cnn_input : ['balourd', 'sain']
Checking directory: ./../../data/05_cnn_input\balourd
Found 246 files in ./../../data/05_cnn_input\balourd
Checking directory: ./../../data/05_cnn_input\sain
Found 551 files in ./../../data/05_cnn_input\sain
Total directories checked: 2
Total image paths collected: 797
Sample image paths: ['./../../data/05_cnn_input\\balourd\\spec_rgb_0551.png', './../../data/05_cnn_input\\balourd\\spec_rgb_0552.png', './../../data/05_cnn_input\\balourd\\spec_rgb_0553.png', './../../data/05_cnn_input\\balourd\\spec_rgb_0554.png', './../../data/05_cnn_input\\balourd\\spec_rgb_0555.png']
Labels distribution: {0: 246, 1: 551}


In [9]:

# Prétraitement des images : charger les images telles quelles (sans redimensionnement ni rognage)
def preprocess_image(image_path):
    try:
        img = load_img(image_path)  # Charger l'image sans redimensionnement
        img_array = img_to_array(img)
        return img_array
    except Exception as e:
        print(f"Erreur lors du traitement de l'image {image_path} : {e}")
        return None

# Débogage : Afficher les informations sur le jeu de données
print(f"Nombre total de chemins d'images trouvés : {len(image_paths)}")
print("Exemples de chemins d'images :", image_paths[:5])
print("Répartition des étiquettes :", {label: labels.tolist().count(label) for label in set(labels)})

# Appliquer le prétraitement à toutes les images
images = np.array([img for img in (preprocess_image(path) for path in image_paths) if img is not None])

# Débogage : Vérifier les images traitées
print(f"Nombre total d'images valides traitées : {len(images)}")
if len(images) > 0:
    print("Dimensions de la première image :", images[0].shape)

# Diviser le jeu de données en ensembles d'entraînement, de validation et de test
if len(images) == 0:
    raise ValueError("Aucune image valide n'a été traitée. Veuillez vérifier le jeu de données.")

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Débogage : Afficher les répartitions des ensembles de données
print(f"Ensemble d'entraînement : {len(X_train)} échantillons")
print(f"Ensemble de validation : {len(X_val)} échantillons")
print(f"Ensemble de test : {len(X_test)} échantillons")

Nombre total de chemins d'images trouvés : 797
Exemples de chemins d'images : ['./../../data/05_cnn_input\\balourd\\spec_rgb_0551.png'
 './../../data/05_cnn_input\\balourd\\spec_rgb_0552.png'
 './../../data/05_cnn_input\\balourd\\spec_rgb_0553.png'
 './../../data/05_cnn_input\\balourd\\spec_rgb_0554.png'
 './../../data/05_cnn_input\\balourd\\spec_rgb_0555.png']
Répartition des étiquettes : {np.int64(0): 246, np.int64(1): 551}
Nombre total d'images valides traitées : 797
Dimensions de la première image : (129, 101, 3)
Ensemble d'entraînement : 478 échantillons
Ensemble de validation : 159 échantillons
Ensemble de test : 160 échantillons


## Construction du modèle CNN

Nous allons définir une architecture de réseau de neurones convolutionnel (CNN) pour traiter les images de spectrogrammes.

In [13]:

# Définir le modèle CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(129, 101, 3)),  # Updated input shape
    BatchNormalization(),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


# model = models.Sequential([
#     layers.InputLayer(input_shape=(img_height, img_width, 3)),

#     layers.Conv2D(32, kernel_size_5_5, activation='relu'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(),

#     layers.Conv2D(64, kernel_size_3_3, activation='relu'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(),

#     layers.Conv2D(128, kernel_size_3_3, activation='relu'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(),

#     layers.Conv2D(256, kernel_size_3_3, activation='relu'), # Nouveau bloc
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(),

#     layers.Flatten(),

#     layers.Dense(256, activation='relu'),
#     layers.Dropout(0.5),
#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.5),

#     # Couche de sortie avec 2 neurones (pour chien et chat) et activation sigmoïde.
#     layers.Dense(1, activation='sigmoid')
# ])


# Afficher le résumé du modèle
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 127, 99, 32)    │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 127, 99, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 61, 47, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 28, 21, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 12, 8, 256)     │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 12, 8, 256)     │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 6, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 6144)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │     1,573,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,996,481 (7.62 MB)

 Trainable params: 1,995,521 (7.61 MB)

 Non-trainable params: 960 (3.75 KB)

## Entraînement du modèle et sauvegarde

Nous allons compiler le modèle, l'entraîner en utilisant les ensembles d'entraînement et de validation, puis enregistrer le modèle entraîné avec une convention de nommage basée sur les performances.

In [14]:

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32
)

# Générer un nom de fichier avec un timestamp et les performances
def generate_model_name(history, test_accuracy):
    val_accuracy = max(history.history['val_accuracy']) * 100
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    return f"{timestamp}_validation_res_{val_accuracy:.2f}_test_set_{test_accuracy:.2f}.h5"

Epoch 1/15


15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 160ms/step - accuracy: 0.6423 - loss: 1.8105 - val_accuracy: 0.7107 - val_loss: 0.8666
Epoch 2/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - accuracy: 0.6799 - loss: 1.2353 - val_accuracy: 0.7107 - val_loss: 1.0936
Epoch 3/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - accuracy: 0.7992 - loss: 0.6654 - val_accuracy: 0.7107 - val_loss: 1.8502
Epoch 4/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 0.8766 - loss: 0.3480 - val_accuracy: 0.7107 - val_loss: 1.1643
Epoch 5/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.8828 - loss: 0.2556 - val_accuracy: 0.7107 - val_loss: 1.2664
Epoch 6/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 0.9519 - loss: 0.1465 - val_accuracy: 0.7107 - val_loss: 1.7342
Epoch 7/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - accuracy: 0.9644 - loss: 0.1091 - val_accuracy: 0.7673 - val_loss: 0.3854
Epoch 8/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - accuracy: 0.9665 - loss: 0.0634 - val_accuracy: 0.9434 - val_

## Évaluation du modèle

Nous allons évaluer le modèle entraîné sur l'ensemble de test et calculer la précision sur cet ensemble.

In [16]:
## Évaluation du modèle
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble de test : {test_accuracy * 100:.2f}%")

if test_accuracy * 100 > 97:
    # Sauvegarder le modèle avec la précision mise à jour
    model.save(generate_model_name(history, test_accuracy * 100))
    print("Modèle sauvegardé avec succès.")
else:
    print("La précision sur l'ensemble de test est inférieure à 98%. Le modèle ne sera pas sauvegardé.")

1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 0.0224

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9750 - loss: 0.1195


Précision sur l'ensemble de test : 97.50%
Modèle sauvegardé avec succès.
